### Model Training and Cross Validation

In [1]:
#load library
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Subset
#load models and other customized component
from models.base_model import BaseModel, MagicModel1, MagicModel2
from utils.model_util import fit, predict, hyp_random_search
from utils.loss_functions import ProbCatCrossEntropyLoss
from utils.performance import categorical_accuracy
from utils.preprocessing import GlobalContrastNormalization, ZCATransformation, zca_whitening_matrix

import pickle

#set gcn transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     GlobalContrastNormalization(1, 10, 1e-6)])


#set gcn zca transform
#calculate ZCA whitening
transform = transforms.Compose(
    [transforms.ToTensor(),
     GlobalContrastNormalization(1, 10, 1e-6)])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=250, shuffle=False, num_workers=0)

data = np.concatenate([d for d, _ in trainloader], axis=0)
zca_mean = torch.from_numpy(np.mean(data, axis=0).flatten().astype(np.float32))
zca_matrix = torch.from_numpy(zca_whitening_matrix(data).astype(np.float32))

#set transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     GlobalContrastNormalization(1, 10, 1e-6),
     ZCATransformation(zca_matrix, zca_mean)])


#set normalization transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [2]:
#set norm zca transform
#calculate ZCA whitening
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=250, shuffle=False, num_workers=0)

data = np.concatenate([d for d, _ in trainloader], axis=0)
zca_mean = torch.from_numpy(np.mean(data, axis=0).flatten().astype(np.float32))
zca_matrix = torch.from_numpy(zca_whitening_matrix(data).astype(np.float32))

#set transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ZCATransformation(zca_matrix, zca_mean)])


Files already downloaded and verified


In [3]:
#get data (_, 3, 32, 32)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
#split validation set from training set
smallset = Subset(trainset, np.arange(200))

train_length = len(trainset)
train_index = np.arange(int(train_length*0.8))
valid_index = np.arange(int(train_length*0.8), train_length)

validset = Subset(trainset, valid_index)
trainset = Subset(trainset, train_index)

smallloader = DataLoader(smallset, batch_size = 4, shuffle=False, num_workers=4, pin_memory =True)
trainloader = DataLoader(trainset, batch_size=250, shuffle=True, num_workers=4, pin_memory =True)
validloader = DataLoader(validset, batch_size=250, shuffle=False, num_workers=4, pin_memory =True)
testloader = DataLoader(testset, batch_size=250, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
#initialize model, optimizer, etc
model = MagicModel1((3, 32, 32), 10)
loss_function = ProbCatCrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3) #, momentum=0.9
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 10, gamma=1)

hyp_search_param = {'lr_low': -6, 'lr_high': -3, 
                    'reg_low': -5, 'reg_high': 0,
                    'lr_step_low': 1, 'lr_step_high': 50,
                    'lr_gamma_low': -2, 'lr_gamma_high': 0}
result = hyp_random_search(MagicModel1, trainloader, validloader, loss_function, epochs = 50, max_count = 60, param=hyp_search_param)

#save hyperparameter search result
with open('hyper1.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
#training
history = fit(model, trainloader, optimizer, loss_function, lr_scheduler, epochs=50, valid_loader = validloader, measure = ['accuracy'], verbose = 1)
#history = fit(model, smallloader, optimizer, loss_function, lr_scheduler, epochs=40, measure = ['accuracy'])

Epoch 0:   1% 1/160 [00:01<03:46,  1.42s/it, training_loss=2.29, train_accuracy=0.104]

/home/jwang75/tmp/workspace/models/base_model.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


Epoch 0: 100% 160/160 [00:50<00:00,  3.17it/s, training_loss=1.6, train_accuracy=0.426, val_loss=1.29, val_accuracy=0.543]
Epoch 1: 100% 160/160 [00:52<00:00,  3.03it/s, training_loss=1.07, train_accuracy=0.632, val_loss=1.26, val_accuracy=0.56]
Epoch 2: 100% 160/160 [00:47<00:00,  3.36it/s, training_loss=0.817, train_accuracy=0.725, val_loss=1.09, val_accuracy=0.623]
Epoch 3: 100% 160/160 [00:50<00:00,  3.17it/s, training_loss=0.567, train_accuracy=0.821, val_loss=0.995, val_accuracy=0.663]
Epoch 4: 100% 160/160 [00:47<00:00,  3.36it/s, training_loss=0.319, train_accuracy=0.913, val_loss=1.43, val_accuracy=0.573]
Epoch 5: 100% 160/160 [00:52<00:00,  3.07it/s, training_loss=0.156, train_accuracy=0.966, val_loss=1.46, val_accuracy=0.589]
Epoch 6: 100% 160/160 [00:50<00:00,  3.15it/s, training_loss=0.0663, train_accuracy=0.991, val_loss=1.38, val_accuracy=0.626]
Epoch 7: 100% 160/160 [00:51<00:00,  3.13it/s, training_loss=0.0346, train_accuracy=0.997, val_loss=1.35, val_accuracy=0.643]
E

#plot learning curve for loss and accuracy
file_name = "norm_zca_relu_model"
epochs = np.arange(50)
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle("Normalization ZCA relu Model")
ax[0].plot(epochs, history['training history'], linestyle='-', color='b', label='training')
ax[0].plot(epochs, history['validation history'], linestyle='-', color='r', label='validation')
ax[0].set_xlabel("epoch")
ax[0].set_ylabel("loss")

ax[1].plot(epochs, history['training accuracy'], linestyle='-', color='b', label='training')
ax[1].plot(epochs, history['validation accuracy'], linestyle='-', color='r', label='validation')
ax[1].set_xlabel("epoch")
ax[1].set_ylabel("accuracy")

ax[1].legend(loc='lower right')
fig.savefig(file_name+".png", format='png', bbox_inches = 'tight', pad_inches=0.05)

In [8]:
#testing
test_predict = predict(model, testloader)

/home/jwang75/tmp/workspace/models/base_model.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


In [9]:
test_labels = np.concatenate([yb for _, yb in testloader])
categorical_accuracy(np.argmax(test_predict, axis=1).flatten(), test_labels)

0.5043